In [1]:
import tensorflow as tf
import cv2
import numpy as np

In [ ]:
cap = cv2.VideoCapture("keanu.mp4")

# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))
# write the flipped frame
#out.write(frame)

#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('.\opencv-master\data\haarcascades\haarcascade_frontalface_alt2.xml')
profile_face_cascade = cv2.CascadeClassifier('.\opencv-master\data\haarcascades\haarcascade_profileface.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade_left = cv2.CascadeClassifier('.\opencv-master\data\haarcascades\haarcascade_lefteye_2splits.xml')
eye_cascade_right = cv2.CascadeClassifier('.\opencv-master\data\haarcascades\haarcascade_righteye_2splits.xml')


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #cv2.rectangle(frame,(15,25),(200,150),(0,0,255),15)
    #cv2.circle(frame,(100,63), 55, (0,255,0), -1)
    #cv2.line(frame,(0,0),(150,150),(255,255,255),15)
    #font = cv2.FONT_HERSHEY_SIMPLEX
    #cv2.putText(frame,'OpenCV Tuts!',(0,130), font, 1, (200,255,155), 2, cv2.LINE_AA)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    profile_faces = profile_face_cascade.detectMultiScale(gray, 1.3, 5)
    np.append(faces,profile_faces)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        eyes = eye_cascade_left.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)   
        
        eyes = eye_cascade_right.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,0,255),2)   
    

        
    # Display the resulting frame
    cv2.imshow('single frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture("keanu.mp4")

# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))
# write the flipped frame
#out.write(frame)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #cv2.rectangle(frame,(15,25),(200,150),(0,0,255),15)
    #cv2.circle(frame,(100,63), 55, (0,255,0), -1)
    #cv2.line(frame,(0,0),(150,150),(255,255,255),15)
    #font = cv2.FONT_HERSHEY_SIMPLEX
    #cv2.putText(frame,'OpenCV Tuts!',(0,130), font, 1, (200,255,155), 2, cv2.LINE_AA)
            
    # Display the resulting frame
    cv2.imshow('single frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
faces

In [5]:
from nms import nms
import pytesseract
net = cv2.dnn.readNet("frozen_east_text_detection.pb")
outputLayers = []
outputLayers.append("feature_fusion/Conv_7/Sigmoid")
outputLayers.append("feature_fusion/concat_3")

cap = cv2.VideoCapture("worcestershire.mp4")

# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))
# write the flipped frame
#out.write(frame)
#cap.set(1,20*60)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    orig = frame.copy()
    (origHeight, origWidth) = frame.shape[:2]
    
    # set the new width and height and then determine the ratio in change
    # for both the width and height
    (newW, newH) = (320, 320)
    ratioWidth = origWidth / float(newW)
    ratioHeight = origHeight / float(newH)
    
    # resize the image and grab the new image dimensions
    frame = cv2.resize(frame, (newW, newH))
    (frameHeight, frameWidth) = frame.shape[:2]    
    
    
    # frame, pixel scaling, input size, mean subtracted, rgbswap, center crop
    blob = cv2.dnn.blobFromImage(frame, 1.0, (frameWidth, frameHeight), (123.68, 116.78, 103.94), True, False)
    

    net.setInput(blob)
    (scores, geometry) = net.forward(outputLayers)
    
    
    confThreshold = 0.5
    nmsThreshold = 0.4
    
    (rects, confidences, baggage) = decode(scores, geometry, confThreshold)
    
    offsets = []
    thetas = []
    for b in baggage:
        offsets.append(b['offset'])
        thetas.append(b['angle'])
    
    functions = [nms.felzenszwalb.nms, nms.fast.nms, nms.malisiewicz.nms]

    polygons = rects2polys(rects, thetas, offsets, ratioWidth, ratioHeight)
    
    for i, function in enumerate(functions):


        indicies = nms.polygons(polygons, confidences, nms_function=function, confidence_threshold=confThreshold,
                                 nsm_threshold=nmsThreshold)


        indicies = np.array(indicies).reshape(-1)
        if(indicies.size > 0):
            drawpolys = np.array(polygons)[indicies]
            drawOn = orig
            for polygon in drawpolys:
                pts = np.array(polygon, np.int32)
                pts = pts.reshape((-1, 1, 2))
                # draw the polygon
                #cv2.polylines(drawOn, [pts], True, (0,255,0), 2)
                pad = 10
                rectMinX = np.min(pts[:,:,0]) - pad
                rectMaxX = np.max(pts[:,:,0]) + pad
                rectMinY = np.min(pts[:,:,1]) - pad
                rectMaxY = np.max(pts[:,:,1]) + pad
                cv2.rectangle(drawOn,(rectMinX,rectMinY),(rectMaxX,rectMaxY),(0,0,255),2)
                roi = drawOn[rectMinY:rectMaxY, rectMinX:rectMaxX]
                config = ("-l eng --oem 1 --psm 7")
                text = pytesseract.image_to_string(roi, config=config)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(drawOn,text,(rectMinX,rectMinY), font, 1, (200,255,155), 2, cv2.LINE_AA)
                
    cv2.imshow("video",drawOn)
        

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
#cv2.destroyAllWindows()

In [ ]:
if(not polygons):
    print("asdf")

In [ ]:
cap.release()
cv2.destroyAllWindows()




In [ ]:
pts[:,:,0]

In [3]:
import numpy as np
import cv2

#
# rects are rectangles encoded as [x, y, w, h] and used predominately by openCV
# boxes are rectangles encoded as [ulx, uly, lrx, lry]
#
#
#
# returns:
#   rects -- an array of [x, y, w, h] that describe rectangles
#   confidences -- an array of floats correstponding to each rectangle in rects
#   baggage -- an array of dictionaries that contain info about each rect including its offset and angle
#
def decode(scores, geometry, confidenceThreshold):
    
    # grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding confidence scores
    (numRows, numCols) = scores.shape[2:4]
    confidences = []    
    rects = [] #(x,y,w,h)
    baggage =[]

    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the geometrical
        # data used to derive potential bounding box coordinates that
        # surround text
        scoresData = scores[0, 0, y]
        dTop =          geometry[0, 0, y]
        dRight =        geometry[0, 1, y]
        dBottom =       geometry[0, 2, y]
        dLeft =         geometry[0, 3, y]
        anglesData =    geometry[0, 4, y]
            
        # loop over the number of columns
        for x in range(0, numCols):
        
            # if our score does not have sufficient probability, ignore it
            if scoresData[x] < confidenceThreshold:
                continue
    
            confidences.append(float(scoresData[x]))
    
            # compute the offset factor as our resulting feature maps will
            # be 4x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
                
            # extract the rotation angle for the prediction and then
            angle = anglesData[x]
                                        
            # offsetX|Y is where the dTop, dRight, dBottom and dLeft are measured from
            # calc the rect corners
            upperRight = (offsetX + dRight[x], offsetY - dTop[x])
            lowerRight = (offsetX + dRight[x], offsetY + dBottom[x])
            upperLeft = (offsetX - dLeft[x], offsetY - dTop[x])
            lowerLeft = (offsetX - dLeft[x], offsetY + dBottom[x])

            rects.append([
                int(upperLeft[0]), # x
                int(upperLeft[1]),  # y
                int(lowerRight[0]-upperLeft[0]), # w
                int(lowerRight[1]-upperLeft[1]) # h
            ])
            
            baggage.append({
                "offset": (offsetX, offsetY),
                "angle": anglesData[x],
                "upperRight": upperRight,
                "lowerRight": lowerRight,
                "upperLeft": upperLeft,
                "lowerLeft": lowerLeft,
                "dTop": dTop[x],
                "dRight": dRight[x],
                "dBottom": dBottom[x],
                "dLeft": dLeft[x]
            })
    
    return (rects, confidences, baggage)
                 
    

In [4]:
import math


def rects2polys(rects, thetas, origins, ratioWidth=1, ratioHeight=1):
    """Convert rectangles (x,y, w, h) into polygons [(x0,y0), (x1, y1), (x2, y2), (x3, y3])

    :param rects: a list of rectangles, each specified as (x, y, w, h)
    :type rects: tuple
    :param thetas: the angle of rotation for each rectangle in radians
    :type theta: list of float
    :param origin: the point to rotate each rectangle around
    :type origin: list of tuple
    :param ratioWidth: optional width scaling factor, default 1.0
    :type ratioWidth: float
    :param ratioHeight: optional height scaling factor, default 1.0
    :type ratioHeight: float
    :return: a list of polygons, each specified by its (x,y) verticies
    :rtype: list
    """
    polygons = []
    for i, box in enumerate(rects):
        upperLeftX = box[0]
        upperLeftY = box[1]
        lowerRightX = box[0] + box[2]
        lowerRightY = box[1] + box[3]

        # scale the bounding box coordinates based on the respective ratios
        upperLeftX = int(upperLeftX * ratioWidth)
        upperLeftY = int(upperLeftY * ratioHeight)
        lowerRightX = int(lowerRightX * ratioWidth)
        lowerRightY = int(lowerRightY * ratioHeight)

        # create an array of the rectangle's verticies
        points = [
            (upperLeftX, upperLeftY),
            (lowerRightX, upperLeftY),
            (lowerRightX, lowerRightY),
            (upperLeftX, lowerRightY)
        ]

        # the offset is the point at which the rectangle is rotated
        rotationPoint = (int(origins[i][0] * ratioWidth), int(origins[i][1] * ratioHeight))

        polygons.append(rotatePoints(points, thetas[i], rotationPoint))

    return polygons


def rotatePoints(points, theta, origin):
    """Rotate the list of points theta radians around origin

    :param points: list of points, each given as (x,y)
    :type points:  tuple
    :param theta: the angle to rotate the points in radians
    :type theta: float
    :param origin: the point about which the points are to be rotated
    :type origin: tuple
    :return: list of rotated points
    :rtype: list
    """
    rotated = []
    for xy in points:
        rotated.append(rotate_around_point(xy, theta, origin))

    return rotated


def rotate_around_point(xy, radians, origin=(0, 0)):
    """Rotate a point around a given point.

    Adapted from `LyleScott/rotate_2d_point.py` <https://gist.github.com/LyleScott/e36e08bfb23b1f87af68c9051f985302>`_

    :param xy: the (x,y) point to rotate
    :type xy: tuple
    :param radians: the angle in radians to rotate
    :type radians: float
    :param origin: the point to rotate around, defaults to (0,0)
    :returns: the rotated point
    """
    x, y = xy
    offset_x, offset_y = origin
    adjusted_x = (x - offset_x)
    adjusted_y = (y - offset_y)
    cos_rad = math.cos(radians)
    sin_rad = math.sin(radians)
    qx = offset_x + cos_rad * adjusted_x + sin_rad * adjusted_y
    qy = offset_y + -sin_rad * adjusted_x + cos_rad * adjusted_y

    return qx, qy